In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import shap
import geopandas as gpd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
plt.rcParams['font.sans-serif'] = ['SimHei']  # 指定默认
plt.rcParams['axes.unicode_minus'] = False
lizi = xr.open_dataset('E:/ISIMIP/bios/his_.nc')
lat_ = lizi['lat'].values
lon_ = lizi['lon'].values
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#对于全球每个点，得到86，4  的shap值
def shap_value(pr,qs,soilmoist,qr,tasmax,tasmin,rlds,snd,evap,gdp,ppl,co2,lai,fbnf):  
    # 生成随机数据
    X = pd.DataFrame({
        'pr': pr,
        'qs':qs,
        'soilmoist': soilmoist,
        'qr': qr,
        'tasmax': tasmax,
        'tasmin': tasmin,
        'rlds': rlds,
        'snd': snd,
        'evap': evap,
        'gdp': gdp,
        'ppl': ppl,
        'co2': co2,
        'lai': lai
    })
    y = pd.DataFrame({
        'fbnf': fbnf
    })
    y = y['fbnf']*1
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    # 训练随机森林模型
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)

    # 使用SHAP库计算特征重要性
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)   #x的shap值
    return shap_values,X

In [ ]:
#将nc文件转为三维数组 86 360 720 ---》360 720 86
def nc_to_time(nc):
    nc = np.nan_to_num(nc)  #nan值转为0
    var = [[[] for _ in range(720)] for _ in range(360)]   #360 720 86
    for lat in range(360):
        for lon in range(720):
            for ti in range(86):
                var[lat][lon].append(nc[ti][lat][lon])
    return var

In [ ]:
#数据标准化
def stand(ary):
    arry = [[[] for _ in range(720)] for _ in range(360)]
    for lat in range(360):
        for lon in range(720):  #10
            poi = ary[lat][lon]
            aaa = np.array(poi).reshape(-1,1)
            scaler = StandardScaler()
            var = scaler.fit_transform(aaa)
            for i in var:
                arry[lat][lon].append(i[0])
    return arry
def stand_ary(ary):
    poi = []
    aaa = np.array(ary).reshape(-1,1)
    scaler = StandardScaler()
    var = scaler.fit_transform(aaa)
    for i in var:
        poi.append(i[0])
    return poi

In [ ]:
#遍历全球每个点，计算shap
li = np.array(lizi['cvegbg-total'][0])
def get_shape(prnc,qsnc,soilmoistnc,qrnc,tasmaxnc,tasminnc,rldsnc,sndnc,evapnc,gdp,ppl,co2,lainc,varnc):
    num_shape = [[] for _ in range(360)]   #得到   n,11
    for lat in range(360):
        print('---------------第'+str(lat)+'行------------------')
        for lon in range(720):
            if np.isnan(li[lat][lon]):
                num_shape[lat].append(np.nan)
            else:
                shapv,X = shap_value(prnc[lat][lon],qsnc[lat][lon],soilmoistnc[lat][lon],qrnc[lat][lon],tasmaxnc[lat][lon],tasminnc[lat][lon],rldsnc[lat][lon],sndnc[lat][lon],evapnc[lat][lon],gdp[lat][lon],ppl[lat][lon],co2,lainc[lat][lon],varnc[lat][lon])  #获取最大相关性的参数索引
                num_shape[lat].append(shapv)
    return num_shape

In [ ]:
### pr,qs,soilmoist,qr,tasmax,tasmin,rlds,snd,evap,landuse_croptotal,ppl
pr_nc = xr.open_dataset('E:/ISIMIP/bios/input_pr/mean/year126_mean.nc')['input_pr']
qs_nc = xr.open_dataset('E:/ISIMIP/bios/qs/mean/year126_mean.nc')['qs']
soilmoist_nc = xr.open_dataset('E:/ISIMIP/bios/soilmoist/mean/year126_mean.nc')['soilmoist']
qr_nc = xr.open_dataset("E:/ISIMIP/1CWatM/year126_mean.nc")['qr']
tasmax_nc = xr.open_dataset('E:/ISIMIP/bios/tasmax/mean/year126_mean.nc')['tasmax']
tasmin_nc = xr.open_dataset('E:/ISIMIP/bios/tasmin/mean/year126_mean.nc')['tasmin']
rlds_nc = xr.open_dataset('E:/ISIMIP/bios/rlds/mean/year126_mean.nc')['rlds']
snd_nc = xr.open_dataset('E:/ISIMIP/bios/snd/mean/year126_mean.nc')['snd']
evap_nc = xr.open_dataset('E:/ISIMIP/bios/evap/mean/year126_mean.nc')['evap']
gdp_nc = xr.open_dataset("E:/ISIMIP/bios/population/gdp_ssp126.nc")['gdp']
ppl_nc = xr.open_dataset("E:/ISIMIP/bios/population/ppl_ssp126.nc")['ppl']
lai_nc = xr.open_dataset('E:/ISIMIP/bios/lai/mean/year126_mean.nc')['lai']
###
fbnf_nc = xr.open_dataset('E:/ISIMIP/bios/fbnf/mean/year126_mean.nc')['fbnf']
#改为360 720 86格式
pr_nnc = nc_to_time(pr_nc)
qs_nnc = nc_to_time(qs_nc)
soilmoist_nnc = nc_to_time(soilmoist_nc)
qr_nnc = nc_to_time(qr_nc)
tasmax_nnc = nc_to_time(tasmax_nc)
tasmin_nnc = nc_to_time(tasmin_nc)
rlds_nnc = nc_to_time(rlds_nc)
snd_nnc = nc_to_time(snd_nc)
evap_nnc = nc_to_time(evap_nc)
gdp_nnc = nc_to_time(gdp_nc)
ppl_nnc = nc_to_time(ppl_nc)
co2 = [399.95, 403.12, 405.75, 408.59, 411.42, 414.23, 417.04, 419.81, 422.5, 425.12, 427.67, 430.17, 432.6, 434.97, 437.29, 439.56, 441.78, 443.93, 445.99, 447.97, 449.87, 451.68, 453.43, 455.09, 456.68, 458.2, 459.65, 461.02, 462.31, 463.54, 464.68, 465.75, 466.75, 467.68, 468.53, 469.31, 470.02, 470.66, 471.25, 471.78, 472.25, 472.66, 473.02, 473.32, 473.56, 473.75, 473.88, 473.96, 474.0, 474.0, 473.96, 473.87, 473.75, 473.58, 473.36, 473.11, 472.81, 472.46, 472.04, 471.56, 471.02, 470.41, 469.75, 469.02, 468.24, 467.39, 466.48, 465.54, 464.56, 463.56, 462.53, 461.47, 460.38, 459.26, 458.1, 456.92, 455.71, 454.5, 453.32, 452.16, 451.02, 449.91, 448.81, 447.73, 446.67, 445.62]
lai_nnc = nc_to_time(lai_nc)
###
fbnf_nnc = nc_to_time(fbnf_nc)
#数据标准化
pr_std = stand(pr_nnc)
qs_std = stand(qs_nnc)
soilmoist_std = stand(soilmoist_nnc)
qr_std = stand(qr_nnc)
tasmax_std = stand(tasmax_nnc)
tasmin_std = stand(tasmin_nnc)
rlds_std = stand(rlds_nnc)
snd_std = stand(snd_nnc)
evap_std = stand(evap_nnc)
ppl_std = stand(ppl_nnc)
gdp_std = stand(gdp_nnc)
co2_std = stand_ary(co2)
lai_std = stand(lai_nnc)
###
fbnf_std = stand(fbnf_nnc)

In [ ]:
##
num_sv = get_shape(pr_std,qs_std,soilmoist_std,qr_std,tasmax_std,tasmin_std,rlds_std,snd_std,evap_std,gdp_std,ppl_std,co2,lai_std,fbnf_std)

In [ ]:
# 计算全球逐像元shap占比
out_shap_percent = [[] for _ in range(360)]
out_shap_abs = [[] for _ in range(360)]
out_shap = [[] for _ in range(360)]
for lat in range(360):
    for lon in range(720):
        if np.isnan(li[lat][lon]):
            out_shap_percent[lat].append([np.nan for _ in range(13)])
            out_shap_abs[lat].append([np.nan for _ in range(13)])
            out_shap[lat].append([np.nan for _ in range(13)])
        else:
            # 第一步：对每个指标（列）在86年内取绝对值并求和，得到长度为13的数组
            abs_sum_per_indicator = np.mean(abs(num_sv[lat][lon]), axis=0)
            out_shap_abs[lat].append(abs_sum_per_indicator)
            out_shap[lat].append(np.mean(num_sv[lat][lon],axis=0))
            # 第二步：计算每个指标的绝对值和占总和的百分比
            percentage_per_indicator = (abs_sum_per_indicator/sum(abs_sum_per_indicator))*100
            out_shap_percent[lat].append(percentage_per_indicator)
out_shap_percent = np.array(out_shap_percent)
out_shap_abs = np.array(out_shap_abs)
out_shap = np.array(out_shap)

In [ ]:
# 创建维度坐标
lat = np.linspace(89.75, -89.75, 360)       # 纬度
lon = np.linspace(-179.75, 179.75, 720)     # 经度
var = np.array(['pr','qs','soilmoist','qr','tasmax','tasmin','rlds','snd','evap','gdp','ppl','co2','lai'])                  # 时间索引或具体时间戳
# 创建 xarray 数据集
ds_persent = xr.Dataset(
    {
        "var_shap": (["lat", "lon", "var"], out_shap_percent)
    },
    coords={
        "lat": lat,
        "lon": lon,
        "var": var
    }
)
ds_shap_abs = xr.Dataset(
    {
        "var_shap": (["lat", "lon", "var"], out_shap_abs)
    },
    coords={
        "lat": lat,
        "lon": lon,
        "var": var
    }
)
ds_shap = xr.Dataset(
    {
        "var_shap": (["lat", "lon", "var"], out_shap)
    },
    coords={
        "lat": lat,
        "lon": lon,
        "var": var
    }
)
# 保存为 NetCDF 文件
ds_persent.to_netcdf("var_shap_percent126.nc")
ds_shap_abs.to_netcdf("var_shap_abs126.nc")
ds_shap.to_netcdf("var_shap126.nc")

In [ ]:
# 分区域计算占比
SREX_shap = np.array([[float(0.0) for _ in range(13)] for _ in range(15)])  #15,13 15SREX,13shap
Srex = [2,6,8,9,12,14,15,17,18,20,21,23,25,26,27]
SREX_MAP = xr.open_dataset("D:/ISIMIP/tif_nc/asdds_nan111.nc")['band_data']
SREX_MAP = SREX_MAP[::-1]

In [ ]:
for lat in range(360):
    print(lat)
    for lon in range(720):
        if np.isnan(SREX_MAP[lat][lon]) or np.all(np.isnan(out_shap_percent[lat][lon])):
            continue
        else:
            if SREX_MAP[lat][lon] == 2:
                SREX_shap[0] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 6:
                SREX_shap[1] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 8:
                SREX_shap[2] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 9:
                SREX_shap[3] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 12:
                SREX_shap[4] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 14:
                SREX_shap[5] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 15:
                SREX_shap[6] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 17:
                SREX_shap[7] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 18:
                SREX_shap[8] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 20:
                SREX_shap[9] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 21:
                SREX_shap[10] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 23:
                SREX_shap[11] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 25:
                SREX_shap[12] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 26:
                SREX_shap[13] += out_shap_percent[lat][lon]
                print('serach success')
            elif SREX_MAP[lat][lon] == 27:
                SREX_shap[14] += out_shap_percent[lat][lon]
                print('serach success')

In [ ]:
SREX_percent = [[] for _ in range(15)]   #[ [13] 15]
for inu in range(len(SREX_shap)):  #
    SREX_percent[inu] = SREX_shap[inu] / (np.sum(SREX_shap[inu]))  #遍历地区，SREX_shap[inu]是单个地区shap[13],
np.round(SREX_percent,3)

In [ ]:
###占比百分数和shap值
sq_shape_qr = [[0 for _ in range(720)] for _ in range(360)]
plt_sq = []
aaaa = []
evappp = []
rldsss = []
taxmaxxx = []
taxminnn = []
for lat in range(360):
    for lon in range(720):
        if np.isnan(li[lat][lon]):
            sq_shape_qr[lat][lon] = np.nan
        else:
            sq = np.mean(abs(num_sv[lat][lon]),axis=0)    #绝对值的均值
            aaa = np.mean(num_sv[lat][lon],axis=0)   #计算每个点上的shap均值
            aaaa.append(aaa[3])
            evappp.append(aaa[8])
            rldsss.append(aaa[6])
            taxmaxxx.append(aaa[4])
            taxminnn.append(aaa[5])
            plt_sq.append(sq)
            if sum(sq) == 0:
                sq_shape_qr[lat][lon] = 0
            else:
                qr_sq = (sq[3]/sum(sq))*100
                sq_shape_qr[lat][lon] = qr_sq
sq_shape_qr = np.array(sq_shape_qr)
plt_sq = np.array(plt_sq)
result111 = np.array(aaaa)
result222 = np.array(evappp)
result333 = np.array(rldsss)
result444 = np.array(taxmaxxx)
result555 = np.array(taxminnn)
np.savetxt('./aaaa/aaaa_126fbnf.csv', result111, delimiter=',')
np.savetxt('./aaaa/evap_126fbnf.csv', result222, delimiter=',')
np.savetxt('./aaaa/rlds_126fbnf.csv', result333, delimiter=',')
np.savetxt('./aaaa/tasmax_126fbnf.csv', result444, delimiter=',')
np.savetxt('./aaaa/tasmin_126fbnf.csv', result555, delimiter=',')

In [ ]:
'''红色: #FF0000  橙色: #FFA500  黄色: #FFFF00  绿色: #008000  蓝色: #0000FF  紫色: #800080  粉色: #FFC0CB 深红色: #8B0000  天蓝色: #87CEEB  深绿色: #006400  
   浅灰色: #D3D3D3  黑色: #000000  白色: #FFFFFF  灰色: #808080  棕色: #A52A2A  金色: #FFD700  银色: #C0C0C0  海洋蓝: #20B2AA  亮紫色: #DDA0DD  青色: #00FFFF
'''
### pr,qs,soilmoist,qr,tasmax,tasmin,rlds,snd,evap,gdp,ppl,lai
pr_num = 0
qs_num = 0
soilmoist_num = 0
qr_num = 0
tasmax_num = 0
tasmin_num = 0
rlds_num = 0
snd_num = 0
evap_num = 0
gdp_num = 0
ppl_num = 0
co2_num = 0
lai_num = 0
num_ = 0
for i in plt_sq:
    pr_num += abs(i[0])  #pr总计数
    qs_num += abs(i[1])  #qs总计数
    soilmoist_num += abs(i[2])  #soilmoist总计数
    qr_num += abs(i[3])  #qr总计数
    tasmax_num += abs(i[4])  #tasmax总计数
    tasmin_num += abs(i[5])  #tasmin总计数
    rlds_num += abs(i[6])  #rlds总计数
    snd_num += abs(i[7])  #snd总计数
    evap_num += abs(i[8])  #evap总计数
    gdp_num += abs(i[9])  #gdp总计数
    ppl_num += abs(i[10])  #ppl总计数
    co2_num += abs(i[11])  
    lai_num += abs(i[12])
    num_ = num_ + abs(i[0]) + abs(i[1]) + abs(i[2]) + abs(i[3]) + abs(i[4]) + abs(i[5]) + abs(i[6]) + abs(i[7]) + abs(i[8]) + abs(i[9]) + abs(i[10]) + abs(i[11]) + abs(i[12])#总计数
pr_s = (pr_num / num_) * 100
qs_s = (qs_num / num_) * 100
soilmoist_s = (soilmoist_num / num_) * 100
qr_s = (qr_num / num_) * 100
tasmax_s = (tasmax_num / num_) * 100
tasmin_s = (tasmin_num / num_) * 100
rlds_s = (rlds_num / num_) * 100
snd_s = (snd_num / num_) * 100
evap_s = (evap_num / num_) * 100
gdp_s = (gdp_num / num_) * 100
ppl_s = (ppl_num / num_) * 100
co2_s = (co2_num / num_) * 100
lai_s = (lai_num / num_) * 100
# 数据
labels = ['pr', 'qs', 'soilmoist', 'qr', 'tasmax', 'tasmin', 'rlds', 'snd', 'evap', 'gdp', 'ppl', 'co2', 'lai']  # 每个部分的标签
sizes = [pr_s, qs_s, soilmoist_s, qr_s, tasmax_s, tasmin_s, rlds_s, snd_s, evap_s, gdp_s, ppl_s, co2_s, lai_s]  # 每个部分的大小
#colors = ['#FF0000', '#0000FF', '#20B2AA', '#008000', '#FFA500', '#FFC0CB', '#800080', '#A52A2A', '#FFD700', '#C0C0C0', '#87CEEB', '#BBCEEB']  # 每个部分的颜色
colors = ['#a3e12e', '#0353c1', '#f6a3b9', '#8c0a2d', '#d4f5a7', '#3b7e5a', '#f89c3d', '#6a2d91', '#c4e3f2', '#e8764b', '#9d1aef', '#5cf7c0', '#2b3a7f']
# 绘制饼状图
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
# 保持饼图为圆形
plt.axis('equal')
plt.title('126下各种影响因素对生物固氮的shap值占比百分数')
# 显示图形
plt.show()
print(sizes)

In [ ]:
out_nc = xr.Dataset() # 创建一个空的Dataset
out_nc['qr%'] = (('lat','lon'), np.array(sq_shape_qr)) # 输出文件的变量名
out_nc['lat'] = (('lat'), lat_)
out_nc['lon'] = (('lon'), lon_)
out_nc['qr%'].attrs['units'] = '%' # 修改单位
out_nc['lat'].attrs['units'] = 'degrees_north' # 配置lat的单位
out_nc['lon'].attrs['units'] = 'degrees_east' # 配置lon的单位
# out_nc['qr%'].plot(figsize=(12,6),vmin=0,vmax=100,cmap='RdBu_r')
out_nc.to_netcdf('E:/ISIMIP/bios/shap/qr_126fbnf_13_mean1.nc') # 输出文件路径